In [ ]:
from Bio.Seq import Seq
from Bio import SeqIO
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
import pandas as pd
import numpy as np
import time
import subprocess, sys, os
import copy
import lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras import optimizers
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping 

import matplotlib.pyplot as plt
%matplotlib inline

print(tf.__version__)

In [ ]:
# data preprocessing

#os.system('for chr in `seq 1 22` X Y; do echo $chr; wget -qO- http://hgdownload.cse.ucsc.edu/goldenpath/hg38/chromosomes/chr$chr.fa.gz | gunzip -c - >> /mnt/HDD8TB/MicroSEC/source/hg38.fa; done')
#os.system('for chr in `seq 1 22` X Y; do echo $chr; wget -qO- http://hgdownload.cse.ucsc.edu/goldenpath/hg19/chromosomes/chr$chr.fa.gz | gunzip -c - >> /mnt/HDD8TB/MicroSEC/source/hg19.fa; done')

#pd.set_option('display.max_columns', 30)
#pd.set_option('display.max_rows', 30)

df = pd.read_excel("/mnt/HDD8TB/MicroSEC/source/MANOSEC_source.xlsx")

sort_order = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10',
              'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20',
              'chr21', 'chr22', 'chrX', 'chrY']
df.Chr = pd.Categorical(df.Chr, categories = sort_order)
df.sort_values(by=['Chr'], inplace=True)
df = df.reset_index()
df['palindrome'] = ""
df['search_seq_A'] = ""
df['search_seq_B'] = ""
df['search_seq_C'] = ""
df['search_seq_D'] = ""
df['rev_comp_seq'] = 0
df['search_hairpin_A'] = ""
df['search_hairpin_B'] = ""
df['search_hairpin_C'] = ""
df['search_hairpin_D'] = ""
df['search_hairpin_E'] = ""
df['possible_hairpin_A_1'] = 0
df['possible_hairpin_A_2'] = 0
df['possible_hairpin_A_3'] = 0
df['possible_hairpin_A_4'] = 0
df['possible_hairpin_A_5'] = 0
df['possible_hairpin_A_6'] = 0
df['possible_hairpin_B_1'] = 0
df['possible_hairpin_B_2'] = 0
df['possible_hairpin_B_3'] = 0
df['possible_hairpin_B_4'] = 0
df['possible_hairpin_B_5'] = 0
df['possible_hairpin_B_6'] = 0
df['possible_hairpin_C_1'] = 0
df['possible_hairpin_C_2'] = 0
df['possible_hairpin_C_3'] = 0
df['possible_hairpin_C_4'] = 0
df['possible_hairpin_C_5'] = 0
df['possible_hairpin_C_6'] = 0
df['possible_hairpin_D_1'] = 0
df['possible_hairpin_D_2'] = 0
df['possible_hairpin_D_3'] = 0
df['possible_hairpin_D_4'] = 0
df['possible_hairpin_D_5'] = 0
df['possible_hairpin_D_6'] = 0
df['possible_hairpin_E_1'] = 0
df['possible_hairpin_E_2'] = 0
df['possible_hairpin_E_3'] = 0
df['possible_hairpin_E_4'] = 0
df['possible_hairpin_E_5'] = 0
df['possible_hairpin_E_6'] = 0
df['score_A_20'] = 0
df['score_A_25'] = 0
df['score_A_30'] = 0
df['score_A_35'] = 0
df['score_A_40'] = 0
df['score_A_45'] = 0
df['score_A_50'] = 0
df['score_A_55'] = 0
df['score_A_60'] = 0
df['score_B_20'] = 0
df['score_B_25'] = 0
df['score_B_30'] = 0
df['score_B_35'] = 0
df['score_B_40'] = 0
df['score_B_45'] = 0
df['score_B_50'] = 0
df['score_B_55'] = 0
df['score_B_60'] = 0
df['score_C_20'] = 0
df['score_C_25'] = 0
df['score_C_30'] = 0
df['score_C_35'] = 0
df['score_C_40'] = 0
df['score_C_45'] = 0
df['score_C_50'] = 0
df['score_C_55'] = 0
df['score_C_60'] = 0
df['score_D_20'] = 0
df['score_D_25'] = 0
df['score_D_30'] = 0
df['score_D_35'] = 0
df['score_D_40'] = 0
df['score_D_45'] = 0
df['score_D_50'] = 0
df['score_D_55'] = 0
df['score_D_60'] = 0
df_working = copy.deepcopy(df)
filename_BLAT = "/mnt/HDD8TB/MicroSEC/source/MANOSEC_BLAT.fa"
f = open(filename_BLAT, 'w', encoding='UTF-8')
records = SeqIO.parse('/mnt/HDD8TB/MicroSEC/source/hg38.fa', 'fasta')
j = 0
for record in records:
    if df_working.shape[0] > 0:
        if record.name == df_working.iloc[0].Chr:
            df_tmp = df_working[df_working['Chr'] == df_working.iloc[0].Chr]
            df_working = df_working[df_working['Chr'] != df_working.iloc[0].Chr]
            for i in range(df_tmp.shape[0]):
                if (j + 1) % 500 == 0:
                    print(str(j + 1) + " / " + str(df.shape[0]))
                ID = str(j)
                pos_start_1 = df_tmp.iloc[i].Pos - 31
                pos_start_2 = df_tmp.iloc[i].Pos + len(df_tmp.iloc[i].Ref) - 1
                pos_end_1 = df_tmp.iloc[i].Pos - 1
                pos_end_2 = df_tmp.iloc[i].Pos + len(df_tmp.iloc[i].Ref) + 29
                seq_tmp = (str(record.seq[pos_start_1:pos_end_1]) + df_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_end_2])).lower()
                f.write('>' + ID + '\n')
                f.write(seq_tmp + '\n')
                df.loc[j, 'palindrome'] = (str(record.seq[pos_end_1 - 200:pos_end_1]) + df_tmp.iloc[i].Ref + str(record.seq[pos_start_2:pos_start_2 + 200])).lower()
                df.loc[j, 'search_seq_A'] = (str(record.seq[pos_end_1 - 3:pos_end_1]) + df_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 3])).upper()
                df.loc[j, 'search_seq_B'] = (str(record.seq[pos_end_1 - 4:pos_end_1]) + df_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 4])).upper()
                df.loc[j, 'search_seq_C'] = (str(record.seq[pos_end_1 - 8:pos_end_1]) + df_tmp.iloc[i].Alt).upper()
                df.loc[j, 'search_seq_D'] = (df_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 8])).upper()
                if str(Seq(df_tmp.iloc[i].Alt).reverse_complement()).upper() == df_tmp.iloc[i].Ref.upper():
                    df.loc[j, 'rev_comp_seq'] = 1
                df.loc[j, 'search_hairpin_A'] = (str(record.seq[pos_end_1 - 4:pos_end_1]) + df_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 4])).upper()
                df.loc[j, 'search_hairpin_B'] = (str(record.seq[pos_end_1 - 6:pos_end_1]) + df_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 6])).upper()
                df.loc[j, 'search_hairpin_C'] = (str(record.seq[pos_end_1 - 8:pos_end_1]) + df_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 8])).upper()
                df.loc[j, 'search_hairpin_D'] = (str(record.seq[pos_end_1 - 10:pos_end_1]) + df_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 10])).upper()
                df.loc[j, 'search_hairpin_E'] = (str(record.seq[pos_end_1 - 12:pos_end_1]) + df_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 12])).upper()
                j = j + 1
print(str(j) + " / " + str(df.shape[0]))
f.close()
os.system('blat -stepSize=5 -repMatch=2253 -minScore=20 -minIdentity=0 /mnt/HDD8TB/MicroSEC/source/hg38.fa /mnt/HDD8TB/MicroSEC/source/MANOSEC_BLAT.fa /mnt/HDD8TB/MicroSEC/source/output.pslx -out=pslx -t=dna -q=dna')
os.system('sed "1,5d" /mnt/HDD8TB/MicroSEC/source/output.pslx > /mnt/HDD8TB/MicroSEC/source/output.tsv')

result_BLAT = pd.read_csv('/mnt/HDD8TB/MicroSEC/source/output.tsv', delimiter='\t', header=None)

for i in range(df.shape[0]):
    if (i + 1) % 500 == 0:
        print(str(i + 1) + " / " + str(df.shape[0]))
    df.loc[i, 'possible_hairpin_A_1'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_A']).reverse_complement()).lower(), 1, -1, -1, -1, score_only = True)
    df.loc[i, 'possible_hairpin_A_2'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_A']).reverse_complement()).lower(), 1, -1, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_A_3'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_A']).reverse_complement()).lower(), 1, -2, -1, -1, score_only = True)
    df.loc[i, 'possible_hairpin_A_4'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_A']).reverse_complement()).lower(), 1, -2, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_A_5'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_A']).reverse_complement()).lower(), 1, -3, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_A_6'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_A']).reverse_complement()).lower(), 1, -2, -3, -1, score_only = True)
    df.loc[i, 'possible_hairpin_B_1'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_B']).reverse_complement()).lower(), 1, -1, -1, -1, score_only = True)
    df.loc[i, 'possible_hairpin_B_2'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_B']).reverse_complement()).lower(), 1, -1, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_B_3'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_B']).reverse_complement()).lower(), 1, -2, -1, -1, score_only = True)
    df.loc[i, 'possible_hairpin_B_4'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_B']).reverse_complement()).lower(), 1, -2, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_B_5'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_B']).reverse_complement()).lower(), 1, -3, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_B_6'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_B']).reverse_complement()).lower(), 1, -2, -3, -1, score_only = True)
    df.loc[i, 'possible_hairpin_C_1'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_C']).reverse_complement()).lower(), 1, -1, -1, -1, score_only = True)
    df.loc[i, 'possible_hairpin_C_2'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_C']).reverse_complement()).lower(), 1, -1, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_C_3'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_C']).reverse_complement()).lower(), 1, -2, -1, -1, score_only = True)
    df.loc[i, 'possible_hairpin_C_4'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_C']).reverse_complement()).lower(), 1, -2, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_C_5'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_C']).reverse_complement()).lower(), 1, -3, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_C_6'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_C']).reverse_complement()).lower(), 1, -2, -3, -1, score_only = True)
    df.loc[i, 'possible_hairpin_D_1'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_D']).reverse_complement()).lower(), 1, -1, -1, -1, score_only = True)
    df.loc[i, 'possible_hairpin_D_2'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_D']).reverse_complement()).lower(), 1, -1, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_D_3'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_D']).reverse_complement()).lower(), 1, -2, -1, -1, score_only = True)
    df.loc[i, 'possible_hairpin_D_4'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_D']).reverse_complement()).lower(), 1, -2, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_D_5'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_D']).reverse_complement()).lower(), 1, -3, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_D_6'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_D']).reverse_complement()).lower(), 1, -2, -3, -1, score_only = True)
    df.loc[i, 'possible_hairpin_E_1'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_E']).reverse_complement()).lower(), 1, -1, -1, -1, score_only = True)
    df.loc[i, 'possible_hairpin_E_2'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_E']).reverse_complement()).lower(), 1, -1, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_E_3'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_E']).reverse_complement()).lower(), 1, -2, -1, -1, score_only = True)
    df.loc[i, 'possible_hairpin_E_4'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_E']).reverse_complement()).lower(), 1, -2, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_E_5'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_E']).reverse_complement()).lower(), 1, -3, -2, -1, score_only = True)
    df.loc[i, 'possible_hairpin_E_6'] = pairwise2.align.localms(df.loc[i,'palindrome'], str(Seq(df.loc[i,'search_hairpin_E']).reverse_complement()).lower(), 1, -2, -3, -1, score_only = True)
    tmp_BLAT = result_BLAT[result_BLAT[9] == i]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[22].str.contains(df.loc[i, 'search_seq_A'].lower())]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[7] < 10]
    if tmp_BLAT.shape[0] > 0:
        for j in range(tmp_BLAT.shape[0]):
            tmp_score = tmp_BLAT.iloc[j,12]
            if tmp_score >= 20:
                df.loc[i, 'score_A_20'] = df.loc[i, 'score_A_20'] + 1
            if tmp_score >= 25:
                df.loc[i, 'score_A_25'] = df.loc[i, 'score_A_25'] + 1
            if tmp_score >= 30:
                df.loc[i, 'score_A_30'] = df.loc[i, 'score_A_30'] + 1
            if tmp_score >= 35:
                df.loc[i, 'score_A_35'] = df.loc[i, 'score_A_35'] + 1
            if tmp_score >= 40:
                df.loc[i, 'score_A_40'] = df.loc[i, 'score_A_40'] + 1
            if tmp_score >= 45:
                df.loc[i, 'score_A_45'] = df.loc[i, 'score_A_45'] + 1
            if tmp_score >= 50:
                df.loc[i, 'score_A_50'] = df.loc[i, 'score_A_50'] + 1
            if tmp_score >= 55:
                df.loc[i, 'score_A_55'] = df.loc[i, 'score_A_55'] + 1
            if tmp_score >= 60:
                df.loc[i, 'score_A_60'] = df.loc[i, 'score_A_60'] + 1
    tmp_BLAT = result_BLAT[result_BLAT[9] == i]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[22].str.contains(df.loc[i, 'search_seq_B'].lower())]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[7] < 10]
    if tmp_BLAT.shape[0] > 0:
        for j in range(tmp_BLAT.shape[0]):
            tmp_score = tmp_BLAT.iloc[j,12]
            if tmp_score >= 20:
                df.loc[i, 'score_B_20'] = df.loc[i, 'score_B_20'] + 1
            if tmp_score >= 25:
                df.loc[i, 'score_B_25'] = df.loc[i, 'score_B_25'] + 1
            if tmp_score >= 30:
                df.loc[i, 'score_B_30'] = df.loc[i, 'score_B_30'] + 1
            if tmp_score >= 35:
                df.loc[i, 'score_B_35'] = df.loc[i, 'score_B_35'] + 1
            if tmp_score >= 40:
                df.loc[i, 'score_B_40'] = df.loc[i, 'score_B_40'] + 1
            if tmp_score >= 45:
                df.loc[i, 'score_B_45'] = df.loc[i, 'score_B_45'] + 1
            if tmp_score >= 50:
                df.loc[i, 'score_B_50'] = df.loc[i, 'score_B_50'] + 1
            if tmp_score >= 55:
                df.loc[i, 'score_B_55'] = df.loc[i, 'score_B_55'] + 1
            if tmp_score >= 60:
                df.loc[i, 'score_B_60'] = df.loc[i, 'score_B_60'] + 1

    tmp_BLAT = result_BLAT[result_BLAT[9] == i]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[22].str.contains(df.loc[i, 'search_seq_C'].lower())]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[7] < 10]
    if tmp_BLAT.shape[0] > 0:
        for j in range(tmp_BLAT.shape[0]):
            tmp_score = tmp_BLAT.iloc[j,12]
            if tmp_score >= 20:
                df.loc[i, 'score_C_20'] = df.loc[i, 'score_C_20'] + 1
            if tmp_score >= 25:
                df.loc[i, 'score_C_25'] = df.loc[i, 'score_C_25'] + 1
            if tmp_score >= 30:
                df.loc[i, 'score_C_30'] = df.loc[i, 'score_C_30'] + 1
            if tmp_score >= 35:
                df.loc[i, 'score_C_35'] = df.loc[i, 'score_C_35'] + 1
            if tmp_score >= 40:
                df.loc[i, 'score_C_40'] = df.loc[i, 'score_C_40'] + 1
            if tmp_score >= 45:
                df.loc[i, 'score_C_45'] = df.loc[i, 'score_C_45'] + 1
            if tmp_score >= 50:
                df.loc[i, 'score_C_50'] = df.loc[i, 'score_C_50'] + 1
            if tmp_score >= 55:
                df.loc[i, 'score_C_55'] = df.loc[i, 'score_C_55'] + 1
            if tmp_score >= 60:
                df.loc[i, 'score_C_60'] = df.loc[i, 'score_C_60'] + 1

    tmp_BLAT = result_BLAT[result_BLAT[9] == i]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[22].str.contains(df.loc[i, 'search_seq_D'].lower())]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[7] < 10]
    if tmp_BLAT.shape[0] > 0:
        for j in range(tmp_BLAT.shape[0]):
            tmp_score = tmp_BLAT.iloc[j,12]
            if tmp_score >= 20:
                df.loc[i, 'score_D_20'] = df.loc[i, 'score_D_20'] + 1
            if tmp_score >= 25:
                df.loc[i, 'score_D_25'] = df.loc[i, 'score_D_25'] + 1
            if tmp_score >= 30:
                df.loc[i, 'score_D_30'] = df.loc[i, 'score_D_30'] + 1
            if tmp_score >= 35:
                df.loc[i, 'score_D_35'] = df.loc[i, 'score_D_35'] + 1
            if tmp_score >= 40:
                df.loc[i, 'score_D_40'] = df.loc[i, 'score_D_40'] + 1
            if tmp_score >= 45:
                df.loc[i, 'score_D_45'] = df.loc[i, 'score_D_45'] + 1
            if tmp_score >= 50:
                df.loc[i, 'score_D_50'] = df.loc[i, 'score_D_50'] + 1
            if tmp_score >= 55:
                df.loc[i, 'score_D_55'] = df.loc[i, 'score_D_55'] + 1
            if tmp_score >= 60:
                df.loc[i, 'score_D_60'] = df.loc[i, 'score_D_60'] + 1

print(str(i + 1) + " / " + str(df.shape[0]))

df.to_excel('/mnt/HDD8TB/MicroSEC/source/MANOSEC_processed.xlsx')

X = df.loc[:, ("%Alt", "rev_comp_seq",
               "possible_hairpin_A_1", "possible_hairpin_A_2", "possible_hairpin_A_3", "possible_hairpin_A_4", "possible_hairpin_A_5", "possible_hairpin_A_6",
               "possible_hairpin_B_1", "possible_hairpin_B_2", "possible_hairpin_B_3", "possible_hairpin_B_4", "possible_hairpin_B_5", "possible_hairpin_B_6",
               "possible_hairpin_C_1", "possible_hairpin_C_2", "possible_hairpin_C_3", "possible_hairpin_C_4", "possible_hairpin_C_5", "possible_hairpin_C_6",
               "possible_hairpin_D_1", "possible_hairpin_D_2", "possible_hairpin_D_3", "possible_hairpin_D_4", "possible_hairpin_D_5", "possible_hairpin_D_6",
               "possible_hairpin_E_1", "possible_hairpin_E_2", "possible_hairpin_E_3", "possible_hairpin_E_4", "possible_hairpin_E_5", "possible_hairpin_E_6",
               "score_A_20", "score_A_25", "score_A_30", "score_A_35", "score_A_40", "score_A_45", "score_A_50", "score_A_55", "score_A_60",
               "score_A_20", "score_B_25", "score_B_30", "score_B_35", "score_B_40", "score_B_45", "score_B_50", "score_B_55", "score_B_60",
               "score_C_20", "score_C_25", "score_C_30", "score_C_35", "score_C_40", "score_C_45", "score_C_50", "score_C_55", "score_C_60",
               "score_D_20", "score_D_25", "score_D_30", "score_D_35", "score_D_40", "score_D_45", "score_D_50", "score_D_55", "score_D_60",
              )]
X["SNV"] = df.loc[:, ("Mut_type")].str.contains("snv").astype(int)
X["DEL"] = df.loc[:, ("Mut_type")].str.contains("del").astype(int)
X["INS"] = df.loc[:, ("Mut_type")].str.contains("ins").astype(int)
X["bases"] = df.loc[:, ("Mut_type")].str.split("-", expand=True)[0].astype(int)

y = (df.loc[:, "msec_filter_1234"]).astype(int)


###########################################

df_MSK = pd.read_excel("/mnt/HDD8TB/MicroSEC/source/mutation_MSK.xlsx")

sort_order = ['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10',
              'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20',
              'chr21', 'chr22', 'chrX', 'chrY']
df_MSK.Chr = pd.Categorical(df_MSK.Chr, categories = sort_order)
df_MSK.sort_values(by=['Chr'], inplace=True)
df_MSK = df_MSK.reset_index()
df_MSK['palindrome'] = ""
df_MSK['search_seq_A'] = ""
df_MSK['search_seq_B'] = ""
df_MSK['search_seq_C'] = ""
df_MSK['search_seq_D'] = ""
df_MSK['rev_comp_seq'] = 0
df_MSK['search_hairpin_A'] = ""
df_MSK['search_hairpin_B'] = ""
df_MSK['search_hairpin_C'] = ""
df_MSK['search_hairpin_D'] = ""
df_MSK['search_hairpin_E'] = ""
df_MSK['possible_hairpin_A_1'] = 0
df_MSK['possible_hairpin_A_2'] = 0
df_MSK['possible_hairpin_A_3'] = 0
df_MSK['possible_hairpin_A_4'] = 0
df_MSK['possible_hairpin_A_5'] = 0
df_MSK['possible_hairpin_A_6'] = 0
df_MSK['possible_hairpin_B_1'] = 0
df_MSK['possible_hairpin_B_2'] = 0
df_MSK['possible_hairpin_B_3'] = 0
df_MSK['possible_hairpin_B_4'] = 0
df_MSK['possible_hairpin_B_5'] = 0
df_MSK['possible_hairpin_B_6'] = 0
df_MSK['possible_hairpin_C_1'] = 0
df_MSK['possible_hairpin_C_2'] = 0
df_MSK['possible_hairpin_C_3'] = 0
df_MSK['possible_hairpin_C_4'] = 0
df_MSK['possible_hairpin_C_5'] = 0
df_MSK['possible_hairpin_C_6'] = 0
df_MSK['possible_hairpin_D_1'] = 0
df_MSK['possible_hairpin_D_2'] = 0
df_MSK['possible_hairpin_D_3'] = 0
df_MSK['possible_hairpin_D_4'] = 0
df_MSK['possible_hairpin_D_5'] = 0
df_MSK['possible_hairpin_D_6'] = 0
df_MSK['possible_hairpin_E_1'] = 0
df_MSK['possible_hairpin_E_2'] = 0
df_MSK['possible_hairpin_E_3'] = 0
df_MSK['possible_hairpin_E_4'] = 0
df_MSK['possible_hairpin_E_5'] = 0
df_MSK['possible_hairpin_E_6'] = 0
df_MSK['score_A_20'] = 0
df_MSK['score_A_25'] = 0
df_MSK['score_A_30'] = 0
df_MSK['score_A_35'] = 0
df_MSK['score_A_40'] = 0
df_MSK['score_A_45'] = 0
df_MSK['score_A_50'] = 0
df_MSK['score_A_55'] = 0
df_MSK['score_A_60'] = 0
df_MSK['score_B_20'] = 0
df_MSK['score_B_25'] = 0
df_MSK['score_B_30'] = 0
df_MSK['score_B_35'] = 0
df_MSK['score_B_40'] = 0
df_MSK['score_B_45'] = 0
df_MSK['score_B_50'] = 0
df_MSK['score_B_55'] = 0
df_MSK['score_B_60'] = 0
df_MSK['score_C_20'] = 0
df_MSK['score_C_25'] = 0
df_MSK['score_C_30'] = 0
df_MSK['score_C_35'] = 0
df_MSK['score_C_40'] = 0
df_MSK['score_C_45'] = 0
df_MSK['score_C_50'] = 0
df_MSK['score_C_55'] = 0
df_MSK['score_C_60'] = 0
df_MSK['score_D_20'] = 0
df_MSK['score_D_25'] = 0
df_MSK['score_D_30'] = 0
df_MSK['score_D_35'] = 0
df_MSK['score_D_40'] = 0
df_MSK['score_D_45'] = 0
df_MSK['score_D_50'] = 0
df_MSK['score_D_55'] = 0
df_MSK['score_D_60'] = 0

df_MSK_working = copy.deepcopy(df_MSK)
filename_BLAT = "/mnt/HDD8TB/MicroSEC/source/MANOSEC_BLAT_MSK.fa"
f = open(filename_BLAT, 'w', encoding='UTF-8')
records_MSK = SeqIO.parse('/mnt/HDD8TB/MicroSEC/source/hg19.fa', 'fasta')
j = 0
for record in records_MSK:
    if df_MSK_working.shape[0] > 0:
        if record.name == df_MSK_working.iloc[0].Chr:
            df_MSK_tmp = df_MSK_working[df_MSK_working['Chr'] == df_MSK_working.iloc[0].Chr]
            df_MSK_working = df_MSK_working[df_MSK_working['Chr'] != df_MSK_working.iloc[0].Chr]
            for i in range(df_MSK_tmp.shape[0]):
                if (j + 1) % 500 == 0:
                    print(str(j + 1) + " / " + str(df_MSK.shape[0]))
                ID = str(j)
                pos_start_1 = df_MSK_tmp.iloc[i].Pos - 31
                pos_start_2 = df_MSK_tmp.iloc[i].Pos + len(df_MSK_tmp.iloc[i].Ref) - 1
                pos_end_1 = df_MSK_tmp.iloc[i].Pos - 1
                pos_end_2 = df_MSK_tmp.iloc[i].Pos + len(df_MSK_tmp.iloc[i].Ref) + 29
                seq_tmp = (str(record.seq[pos_start_1:pos_end_1]) + df_MSK_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_end_2])).lower()
                f.write('>' + ID + '\n')
                f.write(seq_tmp + '\n')
                df_MSK.loc[j, 'palindrome'] = (str(record.seq[pos_end_1 - 200:pos_end_1]) + df_MSK_tmp.iloc[i].Ref + str(record.seq[pos_start_2:pos_start_2 + 200])).lower()
                df_MSK.loc[j, 'search_seq_A'] = (str(record.seq[pos_end_1 - 3:pos_end_1]) + df_MSK_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 3])).upper()
                df_MSK.loc[j, 'search_seq_B'] = (str(record.seq[pos_end_1 - 4:pos_end_1]) + df_MSK_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 4])).upper()
                df_MSK.loc[j, 'search_seq_C'] = (str(record.seq[pos_end_1 - 8:pos_end_1]) + df_MSK_tmp.iloc[i].Alt).upper()
                df_MSK.loc[j, 'search_seq_D'] = (df_MSK_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 8])).upper()
                if str(Seq(df_MSK_tmp.iloc[i].Alt).reverse_complement()).upper() == df_MSK_tmp.iloc[i].Ref.upper():
                    df_MSK.loc[j, 'rev_comp_seq'] = 1
                df_MSK.loc[j, 'search_hairpin_A'] = (str(record.seq[pos_end_1 - 4:pos_end_1]) + df_MSK_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 4])).upper()
                df_MSK.loc[j, 'search_hairpin_B'] = (str(record.seq[pos_end_1 - 6:pos_end_1]) + df_MSK_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 6])).upper()
                df_MSK.loc[j, 'search_hairpin_C'] = (str(record.seq[pos_end_1 - 8:pos_end_1]) + df_MSK_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 8])).upper()
                df_MSK.loc[j, 'search_hairpin_D'] = (str(record.seq[pos_end_1 - 10:pos_end_1]) + df_MSK_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 10])).upper()
                df_MSK.loc[j, 'search_hairpin_E'] = (str(record.seq[pos_end_1 - 12:pos_end_1]) + df_MSK_tmp.iloc[i].Alt + str(record.seq[pos_start_2:pos_start_2 + 12])).upper()

                j = j + 1
print(str(j) + " / " + str(df_MSK.shape[0]))
f.close()
os.system('blat -stepSize=5 -repMatch=2253 -minScore=20 -minIdentity=0 /mnt/HDD8TB/MicroSEC/source/hg19.fa /mnt/HDD8TB/MicroSEC/source/MANOSEC_BLAT_MSK.fa /mnt/HDD8TB/MicroSEC/source/output_MSK.pslx -out=pslx -t=dna -q=dna')
os.system('sed "1,5d" /mnt/HDD8TB/MicroSEC/source/output_MSK.pslx > /mnt/HDD8TB/MicroSEC/source/output_MSK.tsv')

result_BLAT_MSK = pd.read_csv('/mnt/HDD8TB/MicroSEC/source/output_MSK.tsv', delimiter='\t', header=None)

for i in range(df_MSK.shape[0]):
    if (i + 1) % 500 == 0:
        print(str(i + 1) + " / " + str(df_MSK.shape[0]))
    df_MSK.loc[i, 'possible_hairpin_A_1'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_A']).reverse_complement()).lower(), 1, -1, -1, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_A_2'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_A']).reverse_complement()).lower(), 1, -1, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_A_3'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_A']).reverse_complement()).lower(), 1, -2, -1, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_A_4'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_A']).reverse_complement()).lower(), 1, -2, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_A_5'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_A']).reverse_complement()).lower(), 1, -3, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_A_6'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_A']).reverse_complement()).lower(), 1, -2, -3, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_B_1'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_B']).reverse_complement()).lower(), 1, -1, -1, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_B_2'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_B']).reverse_complement()).lower(), 1, -1, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_B_3'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_B']).reverse_complement()).lower(), 1, -2, -1, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_B_4'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_B']).reverse_complement()).lower(), 1, -2, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_B_5'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_B']).reverse_complement()).lower(), 1, -3, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_B_6'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_B']).reverse_complement()).lower(), 1, -2, -3, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_C_1'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_C']).reverse_complement()).lower(), 1, -1, -1, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_C_2'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_C']).reverse_complement()).lower(), 1, -1, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_C_3'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_C']).reverse_complement()).lower(), 1, -2, -1, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_C_4'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_C']).reverse_complement()).lower(), 1, -2, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_C_5'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_C']).reverse_complement()).lower(), 1, -3, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_C_6'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_C']).reverse_complement()).lower(), 1, -2, -3, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_D_1'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_D']).reverse_complement()).lower(), 1, -1, -1, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_D_2'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_D']).reverse_complement()).lower(), 1, -1, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_D_3'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_D']).reverse_complement()).lower(), 1, -2, -1, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_D_4'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_D']).reverse_complement()).lower(), 1, -2, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_D_5'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_D']).reverse_complement()).lower(), 1, -3, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_D_6'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_D']).reverse_complement()).lower(), 1, -2, -3, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_E_1'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_E']).reverse_complement()).lower(), 1, -1, -1, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_E_2'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_E']).reverse_complement()).lower(), 1, -1, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_E_3'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_E']).reverse_complement()).lower(), 1, -2, -1, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_E_4'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_E']).reverse_complement()).lower(), 1, -2, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_E_5'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_E']).reverse_complement()).lower(), 1, -3, -2, -1, score_only = True)
    df_MSK.loc[i, 'possible_hairpin_E_6'] = pairwise2.align.localms(df_MSK.loc[i,'palindrome'], str(Seq(df_MSK.loc[i,'search_hairpin_E']).reverse_complement()).lower(), 1, -2, -3, -1, score_only = True)
    tmp_BLAT = result_BLAT[result_BLAT[9] == i]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[22].str.contains(df_MSK.loc[i, 'search_seq_A'].lower())]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[7] < 10]
    if tmp_BLAT.shape[0] > 0:
        for j in range(tmp_BLAT.shape[0]):
            tmp_score = tmp_BLAT.iloc[j,12]
            if tmp_score >= 20:
                df_MSK.loc[i, 'score_A_20'] = df_MSK.loc[i, 'score_A_20'] + 1
            if tmp_score >= 25:
                df_MSK.loc[i, 'score_A_25'] = df_MSK.loc[i, 'score_A_25'] + 1
            if tmp_score >= 30:
                df_MSK.loc[i, 'score_A_30'] = df_MSK.loc[i, 'score_A_30'] + 1
            if tmp_score >= 35:
                df_MSK.loc[i, 'score_A_35'] = df_MSK.loc[i, 'score_A_35'] + 1
            if tmp_score >= 40:
                df_MSK.loc[i, 'score_A_40'] = df_MSK.loc[i, 'score_A_40'] + 1
            if tmp_score >= 45:
                df_MSK.loc[i, 'score_A_45'] = df_MSK.loc[i, 'score_A_45'] + 1
            if tmp_score >= 50:
                df_MSK.loc[i, 'score_A_50'] = df_MSK.loc[i, 'score_A_50'] + 1
            if tmp_score >= 55:
                df_MSK.loc[i, 'score_A_55'] = df_MSK.loc[i, 'score_A_55'] + 1
            if tmp_score >= 60:
                df_MSK.loc[i, 'score_A_60'] = df_MSK.loc[i, 'score_A_60'] + 1
    tmp_BLAT = result_BLAT[result_BLAT[9] == i]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[22].str.contains(df_MSK.loc[i, 'search_seq_B'].lower())]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[7] < 10]
    if tmp_BLAT.shape[0] > 0:
        for j in range(tmp_BLAT.shape[0]):
            tmp_score = tmp_BLAT.iloc[j,12]
            if tmp_score >= 20:
                df_MSK.loc[i, 'score_B_20'] = df_MSK.loc[i, 'score_B_20'] + 1
            if tmp_score >= 25:
                df_MSK.loc[i, 'score_B_25'] = df_MSK.loc[i, 'score_B_25'] + 1
            if tmp_score >= 30:
                df_MSK.loc[i, 'score_B_30'] = df_MSK.loc[i, 'score_B_30'] + 1
            if tmp_score >= 35:
                df_MSK.loc[i, 'score_B_35'] = df_MSK.loc[i, 'score_B_35'] + 1
            if tmp_score >= 40:
                df_MSK.loc[i, 'score_B_40'] = df_MSK.loc[i, 'score_B_40'] + 1
            if tmp_score >= 45:
                df_MSK.loc[i, 'score_B_45'] = df_MSK.loc[i, 'score_B_45'] + 1
            if tmp_score >= 50:
                df_MSK.loc[i, 'score_B_50'] = df_MSK.loc[i, 'score_B_50'] + 1
            if tmp_score >= 55:
                df_MSK.loc[i, 'score_B_55'] = df_MSK.loc[i, 'score_B_55'] + 1
            if tmp_score >= 60:
                df_MSK.loc[i, 'score_B_60'] = df_MSK.loc[i, 'score_B_60'] + 1

    tmp_BLAT = result_BLAT[result_BLAT[9] == i]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[22].str.contains(df_MSK.loc[i, 'search_seq_C'].lower())]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[7] < 10]
    if tmp_BLAT.shape[0] > 0:
        for j in range(tmp_BLAT.shape[0]):
            tmp_score = tmp_BLAT.iloc[j,12]
            if tmp_score >= 20:
                df_MSK.loc[i, 'score_C_20'] = df_MSK.loc[i, 'score_C_20'] + 1
            if tmp_score >= 25:
                df_MSK.loc[i, 'score_C_25'] = df_MSK.loc[i, 'score_C_25'] + 1
            if tmp_score >= 30:
                df_MSK.loc[i, 'score_C_30'] = df_MSK.loc[i, 'score_C_30'] + 1
            if tmp_score >= 35:
                df_MSK.loc[i, 'score_C_35'] = df_MSK.loc[i, 'score_C_35'] + 1
            if tmp_score >= 40:
                df_MSK.loc[i, 'score_C_40'] = df_MSK.loc[i, 'score_C_40'] + 1
            if tmp_score >= 45:
                df_MSK.loc[i, 'score_C_45'] = df_MSK.loc[i, 'score_C_45'] + 1
            if tmp_score >= 50:
                df_MSK.loc[i, 'score_C_50'] = df_MSK.loc[i, 'score_C_50'] + 1
            if tmp_score >= 55:
                df_MSK.loc[i, 'score_C_55'] = df_MSK.loc[i, 'score_C_55'] + 1
            if tmp_score >= 60:
                df_MSK.loc[i, 'score_C_60'] = df_MSK.loc[i, 'score_C_60'] + 1

    tmp_BLAT = result_BLAT[result_BLAT[9] == i]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[22].str.contains(df_MSK.loc[i, 'search_seq_D'].lower())]
    tmp_BLAT = tmp_BLAT[tmp_BLAT[7] < 10]
    if tmp_BLAT.shape[0] > 0:
        for j in range(tmp_BLAT.shape[0]):
            tmp_score = tmp_BLAT.iloc[j,12]
            if tmp_score >= 20:
                df_MSK.loc[i, 'score_D_20'] = df_MSK.loc[i, 'score_D_20'] + 1
            if tmp_score >= 25:
                df_MSK.loc[i, 'score_D_25'] = df_MSK.loc[i, 'score_D_25'] + 1
            if tmp_score >= 30:
                df_MSK.loc[i, 'score_D_30'] = df_MSK.loc[i, 'score_D_30'] + 1
            if tmp_score >= 35:
                df_MSK.loc[i, 'score_D_35'] = df_MSK.loc[i, 'score_D_35'] + 1
            if tmp_score >= 40:
                df_MSK.loc[i, 'score_D_40'] = df_MSK.loc[i, 'score_D_40'] + 1
            if tmp_score >= 45:
                df_MSK.loc[i, 'score_D_45'] = df_MSK.loc[i, 'score_D_45'] + 1
            if tmp_score >= 50:
                df_MSK.loc[i, 'score_D_50'] = df_MSK.loc[i, 'score_D_50'] + 1
            if tmp_score >= 55:
                df_MSK.loc[i, 'score_D_55'] = df_MSK.loc[i, 'score_D_55'] + 1
            if tmp_score >= 60:
                df_MSK.loc[i, 'score_D_60'] = df_MSK.loc[i, 'score_D_60'] + 1

print(str(i + 1) + " / " + str(df_MSK.shape[0]))

df_MSK.to_excel('/mnt/HDD8TB/MicroSEC/source/MANOSEC_processed_MSK.xlsx')

X_MSK = df_MSK.loc[:, ("%Alt", "rev_comp_seq",
               "possible_hairpin_A_1", "possible_hairpin_A_2", "possible_hairpin_A_3", "possible_hairpin_A_4", "possible_hairpin_A_5", "possible_hairpin_A_6",
               "possible_hairpin_B_1", "possible_hairpin_B_2", "possible_hairpin_B_3", "possible_hairpin_B_4", "possible_hairpin_B_5", "possible_hairpin_B_6",
               "possible_hairpin_C_1", "possible_hairpin_C_2", "possible_hairpin_C_3", "possible_hairpin_C_4", "possible_hairpin_C_5", "possible_hairpin_C_6",
               "possible_hairpin_D_1", "possible_hairpin_D_2", "possible_hairpin_D_3", "possible_hairpin_D_4", "possible_hairpin_D_5", "possible_hairpin_D_6",
               "possible_hairpin_E_1", "possible_hairpin_E_2", "possible_hairpin_E_3", "possible_hairpin_E_4", "possible_hairpin_E_5", "possible_hairpin_E_6",
               "score_A_20", "score_A_25", "score_A_30", "score_A_35", "score_A_40", "score_A_45", "score_A_50", "score_A_55", "score_A_60",
               "score_A_20", "score_B_25", "score_B_30", "score_B_35", "score_B_40", "score_B_45", "score_B_50", "score_B_55", "score_B_60",
               "score_C_20", "score_C_25", "score_C_30", "score_C_35", "score_C_40", "score_C_45", "score_C_50", "score_C_55", "score_C_60",
               "score_D_20", "score_D_25", "score_D_30", "score_D_35", "score_D_40", "score_D_45", "score_D_50", "score_D_55", "score_D_60",
              )]

X_MSK["SNV"] = df_MSK.loc[:, ("Mut_type")].str.contains("snv").astype(int)
X_MSK["DEL"] = df_MSK.loc[:, ("Mut_type")].str.contains("del").astype(int)
X_MSK["INS"] = df_MSK.loc[:, ("Mut_type")].str.contains("ins").astype(int)
X_MSK["bases"] = df_MSK.loc[:, ("Mut_type")].str.split("-", expand=True)[0].astype(int)



In [ ]:
df = pd.read_excel('/mnt/HDD8TB/MicroSEC/source/MANOSEC_processed.xlsx')

X = df.loc[:, ("%Alt", "rev_comp_seq",
               "possible_hairpin_A_1", "possible_hairpin_A_2", "possible_hairpin_A_3", "possible_hairpin_A_4", "possible_hairpin_A_5", "possible_hairpin_A_6",
               "possible_hairpin_B_1", "possible_hairpin_B_2", "possible_hairpin_B_3", "possible_hairpin_B_4", "possible_hairpin_B_5", "possible_hairpin_B_6",
               "possible_hairpin_C_1", "possible_hairpin_C_2", "possible_hairpin_C_3", "possible_hairpin_C_4", "possible_hairpin_C_5", "possible_hairpin_C_6",
               "possible_hairpin_D_1", "possible_hairpin_D_2", "possible_hairpin_D_3", "possible_hairpin_D_4", "possible_hairpin_D_5", "possible_hairpin_D_6",
               "possible_hairpin_E_1", "possible_hairpin_E_2", "possible_hairpin_E_3", "possible_hairpin_E_4", "possible_hairpin_E_5", "possible_hairpin_E_6",
               "score_A_20", "score_A_25", "score_A_30", "score_A_35", "score_A_40", "score_A_45", "score_A_50", "score_A_55", "score_A_60",
               "score_B_20", "score_B_25", "score_B_30", "score_B_35", "score_B_40", "score_B_45", "score_B_50", "score_B_55", "score_B_60",
               "score_C_20", "score_C_25", "score_C_30", "score_C_35", "score_C_40", "score_C_45", "score_C_50", "score_C_55", "score_C_60",
               "score_D_20", "score_D_25", "score_D_30", "score_D_35", "score_D_40", "score_D_45", "score_D_50", "score_D_55", "score_D_60",
              )]
X["SNV"] = df.loc[:, ("Mut_type")].str.contains("snv").astype(int)
X["DEL"] = df.loc[:, ("Mut_type")].str.contains("del").astype(int)
X["INS"] = df.loc[:, ("Mut_type")].str.contains("ins").astype(int)
X["bases"] = df.loc[:, ("Mut_type")].str.split("-", expand=True)[0].astype(int)

y = (df.loc[:, "msec_filter_1234"]).astype(int)


df_MSK = pd.read_excel('/mnt/HDD8TB/MicroSEC/source/MANOSEC_processed_MSK.xlsx')

X_MSK = df_MSK.loc[:, ("%Alt", "rev_comp_seq",
               "possible_hairpin_A_1", "possible_hairpin_A_2", "possible_hairpin_A_3", "possible_hairpin_A_4", "possible_hairpin_A_5", "possible_hairpin_A_6",
               "possible_hairpin_B_1", "possible_hairpin_B_2", "possible_hairpin_B_3", "possible_hairpin_B_4", "possible_hairpin_B_5", "possible_hairpin_B_6",
               "possible_hairpin_C_1", "possible_hairpin_C_2", "possible_hairpin_C_3", "possible_hairpin_C_4", "possible_hairpin_C_5", "possible_hairpin_C_6",
               "possible_hairpin_D_1", "possible_hairpin_D_2", "possible_hairpin_D_3", "possible_hairpin_D_4", "possible_hairpin_D_5", "possible_hairpin_D_6",
               "possible_hairpin_E_1", "possible_hairpin_E_2", "possible_hairpin_E_3", "possible_hairpin_E_4", "possible_hairpin_E_5", "possible_hairpin_E_6",
               "score_A_20", "score_A_25", "score_A_30", "score_A_35", "score_A_40", "score_A_45", "score_A_50", "score_A_55", "score_A_60",
               "score_B_20", "score_B_25", "score_B_30", "score_B_35", "score_B_40", "score_B_45", "score_B_50", "score_B_55", "score_B_60",
               "score_C_20", "score_C_25", "score_C_30", "score_C_35", "score_C_40", "score_C_45", "score_C_50", "score_C_55", "score_C_60",
               "score_D_20", "score_D_25", "score_D_30", "score_D_35", "score_D_40", "score_D_45", "score_D_50", "score_D_55", "score_D_60",
              )]

X_MSK["SNV"] = df_MSK.loc[:, ("Mut_type")].str.contains("snv").astype(int)
X_MSK["DEL"] = df_MSK.loc[:, ("Mut_type")].str.contains("del").astype(int)
X_MSK["INS"] = df_MSK.loc[:, ("Mut_type")].str.contains("ins").astype(int)
X_MSK["bases"] = df_MSK.loc[:, ("Mut_type")].str.split("-", expand=True)[0].astype(int)


In [ ]:
# Logistic regression
# 5-fold cross validation
# Training:Validation:Test = 3:1:1

X_LR = np.array(X)
y_LR = np.array(y)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_pred_total_LR = []
y_proba_total_LR = []
y_true_LR = []
for train, test in skf.split(X_LR, y_LR):
    X_train = X_LR[train]
    X_test = X_LR[test]
    y_train = y_LR[train]
    y_test = y_LR[test]
    #X_train, X_valid, y_train, y_valid = train_test_split(X_tv, y_tv, shuffle=True, test_size=0.25, stratify=y_tv, random_state=100)

    search_params = [
        {'solver': ['liblinear', 'saga'],
         'penalty':['l1', 'l2'],
         'C': [0.1, 1, 10, 100],
         'random_state': [2525]},
        {'solver': ['newton-cg', 'sag', 'lbfgs' ],
         'penalty':['l2'],
         'C': [0.1, 1, 10, 100],
         'random_state': [2525]},
    ]


    clf = GridSearchCV(LogisticRegression(),
                      search_params,
                      cv=4,
                      verbose=False,
                      scoring='neg_log_loss',
                      n_jobs=-1)
    clf.fit(X_train, y_train)

    print(clf.best_estimator_)
    print(f"acc: {clf.score(X_test, y_test)}")
    y_pred = clf.predict(X_test)
    y_proba = clf.predict_proba(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1])
    roc_auc = auc(fpr, tpr)
    print ("AUC curve : %f" % roc_auc)
    plt.figure(figsize=(4,4))
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve: AUC=%0.2f' % roc_auc)
    plt.legend(loc="lower right")
    plt.show()
    y_pred_total_LR = np.concatenate([y_pred_total_LR, y_pred])
    y_proba_total_LR = np.concatenate([y_proba_total_LR, y_proba[:, 1]])
    y_true_LR = np.concatenate([y_true_LR, y_test])

fpr, tpr, thresholds = roc_curve(y_true_LR, y_proba_total_LR)
roc_auc = auc(fpr, tpr)
print ("AUC curve : %f" % roc_auc)
plt.figure(figsize=(4,4))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve: AUC=%0.2f' % roc_auc)
plt.legend(loc="lower right")
plt.show()

y_pred_temp_25 = copy.deepcopy(y_proba_total_LR)
for i in range(len(y_proba_total_LR)):
    if y_proba_total_LR[i]>=0.25:
        y_pred_temp_25[i]=1
    else:
        y_pred_temp_25[i]=0

y_pred_temp_50 = copy.deepcopy(y_proba_total_LR)
for i in range(len(y_proba_total_LR)):
    if y_proba_total_LR[i]>=0.5:
        y_pred_temp_50[i]=1
    else:
        y_pred_temp_50[i]=0

y_pred_temp_75 = copy.deepcopy(y_proba_total_LR)
for i in range(len(y_proba_total_LR)):
    if y_proba_total_LR[i]>=0.75:
        y_pred_temp_75[i]=1
    else:
        y_pred_temp_75[i]=0

y_pred_temp_90 = copy.deepcopy(y_proba_total_LR)
for i in range(len(y_proba_total_LR)):
    if y_proba_total_LR[i]>=0.9:
        y_pred_temp_90[i]=1
    else:
        y_pred_temp_90[i]=0

y_pred_temp_95 = copy.deepcopy(y_proba_total_LR)
for i in range(len(y_proba_total_LR)):
    if y_proba_total_LR[i]>=0.95:
        y_pred_temp_95[i]=1
    else:
        y_pred_temp_95[i]=0

y_pred_temp_975 = copy.deepcopy(y_proba_total_LR)
for i in range(len(y_proba_total_LR)):
    if y_proba_total_LR[i]>=0.975:
        y_pred_temp_975[i]=1
    else:
        y_pred_temp_975[i]=0

y_pred_temp_99 = copy.deepcopy(y_proba_total_LR)
for i in range(len(y_proba_total_LR)):
    if y_proba_total_LR[i]>=0.99:
        y_pred_temp_99[i]=1
    else:
        y_pred_temp_99[i]=0

conf_matrix_LR = confusion_matrix(y_pred_temp_25, y_true_LR)
conf_matrix_LR = np.concatenate([conf_matrix_LR, confusion_matrix(y_pred_temp_50, y_true_LR)])
conf_matrix_LR = np.concatenate([conf_matrix_LR, confusion_matrix(y_pred_temp_75, y_true_LR)])
conf_matrix_LR = np.concatenate([conf_matrix_LR, confusion_matrix(y_pred_temp_90, y_true_LR)])
conf_matrix_LR = np.concatenate([conf_matrix_LR, confusion_matrix(y_pred_temp_95, y_true_LR)])
conf_matrix_LR = np.concatenate([conf_matrix_LR, confusion_matrix(y_pred_temp_975, y_true_LR)])
conf_matrix_LR = np.concatenate([conf_matrix_LR, confusion_matrix(y_pred_temp_99, y_true_LR)])

conf_table_LR = pd.DataFrame(conf_matrix_LR, index=['LR_predicted_25 mutation', 'LR_predicted_25 artifact', 
                                              'LR_predicted_50 mutation', 'LR_predicted_50 artifact',
                                              'LR_predicted_75 mutation', 'LR_predicted_75 artifact',
                                              'LR_predicted_90 mutation', 'LR_predicted_90 artifact',
                                              'LR_predicted_95 mutation', 'LR_predicted_95 artifact',
                                              'LR_predicted_97.5 mutation', 'LR_predicted_97.5 artifact',
                                              'LR_predicted_99 mutation', 'LR_predicted_99 artifact'],
                                       columns=['MicroSEC mutation', 'MicroSEC artifact'])

conf_table_LR

In [ ]:
# Random forest classifier
# 5-fold cross validation
# Training:Validation:Test = 3:1:1

X_RF = np.array(X)
y_RF = np.array(y)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_pred_total = []
y_proba_total = []
y_true = []
for train, test in skf.split(X_RF, y_RF):
    X_train = X_RF[train]
    X_test = X_RF[test]
    y_train = y_RF[train]
    y_test = y_RF[test]
    #X_train, X_valid, y_train, y_valid = train_test_split(X_tv, y_tv, shuffle=True, test_size=0.25, stratify=y_tv, random_state=100)

    search_params = {
        'n_estimators'      : [10, 15, 20, 30],
        'max_features'      : [6, 8, 10, 14, 18],
        'random_state'      : [2525],
        'min_samples_split' : [2, 3, 5, 10],
        'max_depth'         : [20, 25, 30]
    }


    gs = GridSearchCV(RFC(),
                      search_params,
                      cv=4,
                      verbose=False,
                      scoring='neg_log_loss',
                      n_jobs=-1)
    gs.fit(X_train, y_train)

    print(gs.best_estimator_)
    print(f"acc: {gs.score(X_test, y_test)}")
    y_pred = gs.predict(X_test)
    y_proba = gs.predict_proba(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1])
    roc_auc = auc(fpr, tpr)
    print ("AUC curve : %f" % roc_auc)
    plt.figure(figsize=(4,4))
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve: AUC=%0.2f' % roc_auc)
    plt.legend(loc="lower right")
    plt.show()
    y_pred_total = np.concatenate([y_pred_total, y_pred])
    y_proba_total = np.concatenate([y_proba_total, y_proba[:, 1]])
    y_true = np.concatenate([y_true, y_test])

fpr, tpr, thresholds = roc_curve(y_true, y_proba_total)
roc_auc = auc(fpr, tpr)
print ("AUC curve : %f" % roc_auc)
plt.figure(figsize=(4,4))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve: AUC=%0.2f' % roc_auc)
plt.legend(loc="lower right")
plt.show()

y_pred_temp_25 = copy.deepcopy(y_proba_total)
for i in range(len(y_proba_total)):
    if y_proba_total[i]>=0.25:
        y_pred_temp_25[i]=1
    else:
        y_pred_temp_25[i]=0

y_pred_temp_50 = copy.deepcopy(y_proba_total)
for i in range(len(y_proba_total)):
    if y_proba_total[i]>=0.5:
        y_pred_temp_50[i]=1
    else:
        y_pred_temp_50[i]=0

y_pred_temp_75 = copy.deepcopy(y_proba_total)
for i in range(len(y_proba_total)):
    if y_proba_total[i]>=0.75:
        y_pred_temp_75[i]=1
    else:
        y_pred_temp_75[i]=0

y_pred_temp_90 = copy.deepcopy(y_proba_total)
for i in range(len(y_proba_total)):
    if y_proba_total[i]>=0.9:
        y_pred_temp_90[i]=1
    else:
        y_pred_temp_90[i]=0

y_pred_temp_95 = copy.deepcopy(y_proba_total)
for i in range(len(y_proba_total)):
    if y_proba_total[i]>=0.95:
        y_pred_temp_95[i]=1
    else:
        y_pred_temp_95[i]=0

y_pred_temp_975 = copy.deepcopy(y_proba_total)
for i in range(len(y_proba_total)):
    if y_proba_total[i]>=0.975:
        y_pred_temp_975[i]=1
    else:
        y_pred_temp_975[i]=0

y_pred_temp_99 = copy.deepcopy(y_proba_total)
for i in range(len(y_proba_total)):
    if y_proba_total[i]>=0.99:
        y_pred_temp_99[i]=1
    else:
        y_pred_temp_99[i]=0

conf_matrix = confusion_matrix(y_pred_temp_25, y_true)
conf_matrix = np.concatenate([conf_matrix, confusion_matrix(y_pred_temp_50, y_true)])
conf_matrix = np.concatenate([conf_matrix, confusion_matrix(y_pred_temp_75, y_true)])
conf_matrix = np.concatenate([conf_matrix, confusion_matrix(y_pred_temp_90, y_true)])
conf_matrix = np.concatenate([conf_matrix, confusion_matrix(y_pred_temp_95, y_true)])
conf_matrix = np.concatenate([conf_matrix, confusion_matrix(y_pred_temp_975, y_true)])
conf_matrix = np.concatenate([conf_matrix, confusion_matrix(y_pred_temp_99, y_true)])

conf_table = pd.DataFrame(conf_matrix, index=['RF_predicted_25 mutation', 'RF_predicted_25 artifact', 
                                              'RF_predicted_50 mutation', 'RF_predicted_50 artifact',
                                              'RF_predicted_75 mutation', 'RF_predicted_75 artifact',
                                              'RF_predicted_90 mutation', 'RF_predicted_90 artifact',
                                              'RF_predicted_95 mutation', 'RF_predicted_95 artifact',
                                              'RF_predicted_97.5 mutation', 'RF_predicted_97.5 artifact',
                                              'RF_predicted_99 mutation', 'RF_predicted_99 artifact'],
                                       columns=['MicroSEC mutation', 'MicroSEC artifact'])

conf_table

In [ ]:
# Decision tree cllasifier and ADABoost
# 5-fold cross validation
# Training:Validation:Test = 3:1:1

X_DT = np.array(X)
y_DT = np.array(y)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_pred_total_DT = []
y_proba_total_DT = []
y_true_DT = []

y_pred_total_ADA = []
y_proba_total_ADA = []
y_true_ADA = []

for train, test in skf.split(X_DT, y_DT):
    X_train = X_DT[train]
    X_test = X_DT[test]
    y_train = y_DT[train]
    y_test = y_DT[test]
    #X_train, X_valid, y_train, y_valid = train_test_split(X_tv, y_tv, shuffle=True, test_size=0.25, stratify=y_tv, random_state=100)

    search_params = {"criterion": ["gini", "entropy"],
                     "splitter": ["best", "random"],
                     "max_depth": [i for i in range(5, 11)],
                     "min_samples_split": [i for i in range(2, 11)],
                     "min_samples_leaf": [i for i in range(2, 11)],
                     "random_state": [2525]
                    }

    dt = GridSearchCV(DecisionTreeClassifier(),
                      search_params,
                      cv=4,
                      verbose=False,
                      n_jobs=-1)
    dt.fit(X_train, y_train)
    criterion_dt, max_depth_dt, min_samples_leaf_dt, min_samples_split_dt, random_state_dt, splitter_dt = dt.best_params_.values()
    ada = AdaBoostClassifier(DecisionTreeClassifier(criterion=criterion_dt, max_depth=max_depth_dt, min_samples_leaf=min_samples_leaf_dt, random_state=random_state_dt, splitter=splitter_dt))
    ada.fit(X_train, y_train)

    print(dt.best_estimator_)
    print(f"acc: {dt.score(X_test, y_test)}")
    print(f"acc: {ada.score(X_test, y_test)}")

    y_pred = dt.predict(X_test)
    y_proba = dt.predict_proba(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1])
    roc_auc = auc(fpr, tpr)
    print ("AUC curve : %f" % roc_auc)
    plt.figure(figsize=(4,4))
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve: AUC=%0.2f' % roc_auc)
    plt.legend(loc="lower right")
    plt.show()
    y_pred_total_DT = np.concatenate([y_pred_total_DT, y_pred])
    y_proba_total_DT = np.concatenate([y_proba_total_DT, y_proba[:, 1]])
    y_true_DT = np.concatenate([y_true_DT, y_test])

    y_pred = ada.predict(X_test)
    y_proba = ada.predict_proba(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1])
    roc_auc = auc(fpr, tpr)
    print ("AUC curve : %f" % roc_auc)
    plt.figure(figsize=(4,4))
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve: AUC=%0.2f' % roc_auc)
    plt.legend(loc="lower right")
    plt.show()
    y_pred_total_ADA = np.concatenate([y_pred_total_ADA, y_pred])
    y_proba_total_ADA = np.concatenate([y_proba_total_ADA, y_proba[:, 1]])
    y_true_ADA = np.concatenate([y_true_ADA, y_test])


fpr, tpr, thresholds = roc_curve(y_true_DT, y_proba_total_DT)
roc_auc = auc(fpr, tpr)
print ("AUC curve : %f" % roc_auc)
plt.figure(figsize=(4,4))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve: AUC=%0.2f' % roc_auc)
plt.legend(loc="lower right")
plt.show()

fpr, tpr, thresholds = roc_curve(y_true_ADA, y_proba_total_ADA)
roc_auc = auc(fpr, tpr)
print ("AUC curve : %f" % roc_auc)
plt.figure(figsize=(4,4))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve: AUC=%0.2f' % roc_auc)
plt.legend(loc="lower right")
plt.show()


y_pred_temp_25 = copy.deepcopy(y_proba_total_DT)
for i in range(len(y_proba_total_DT)):
    if y_proba_total_DT[i]>=0.25:
        y_pred_temp_25[i]=1
    else:
        y_pred_temp_25[i]=0

y_pred_temp_50 = copy.deepcopy(y_proba_total_DT)
for i in range(len(y_proba_total_DT)):
    if y_proba_total_DT[i]>=0.5:
        y_pred_temp_50[i]=1
    else:
        y_pred_temp_50[i]=0

y_pred_temp_75 = copy.deepcopy(y_proba_total_DT)
for i in range(len(y_proba_total_DT)):
    if y_proba_total_DT[i]>=0.75:
        y_pred_temp_75[i]=1
    else:
        y_pred_temp_75[i]=0

y_pred_temp_90 = copy.deepcopy(y_proba_total_DT)
for i in range(len(y_proba_total_DT)):
    if y_proba_total_DT[i]>=0.9:
        y_pred_temp_90[i]=1
    else:
        y_pred_temp_90[i]=0

y_pred_temp_95 = copy.deepcopy(y_proba_total_DT)
for i in range(len(y_proba_total_DT)):
    if y_proba_total_DT[i]>=0.95:
        y_pred_temp_95[i]=1
    else:
        y_pred_temp_95[i]=0

y_pred_temp_975 = copy.deepcopy(y_proba_total_DT)
for i in range(len(y_proba_total_DT)):
    if y_proba_total_DT[i]>=0.975:
        y_pred_temp_975[i]=1
    else:
        y_pred_temp_975[i]=0

y_pred_temp_99 = copy.deepcopy(y_proba_total_DT)
for i in range(len(y_proba_total_DT)):
    if y_proba_total_DT[i]>=0.99:
        y_pred_temp_99[i]=1
    else:
        y_pred_temp_99[i]=0

conf_matrix_DT = confusion_matrix(y_pred_temp_25, y_true_DT)
conf_matrix_DT = np.concatenate([conf_matrix_DT, confusion_matrix(y_pred_temp_50, y_true_DT)])
conf_matrix_DT = np.concatenate([conf_matrix_DT, confusion_matrix(y_pred_temp_75, y_true_DT)])
conf_matrix_DT = np.concatenate([conf_matrix_DT, confusion_matrix(y_pred_temp_90, y_true_DT)])
conf_matrix_DT = np.concatenate([conf_matrix_DT, confusion_matrix(y_pred_temp_95, y_true_DT)])
conf_matrix_DT = np.concatenate([conf_matrix_DT, confusion_matrix(y_pred_temp_975, y_true_DT)])
conf_matrix_DT = np.concatenate([conf_matrix_DT, confusion_matrix(y_pred_temp_99, y_true_DT)])

conf_table_DT = pd.DataFrame(conf_matrix_DT, index=['DT_predicted_25 mutation', 'DT_predicted_25 artifact', 
                                              'DT_predicted_50 mutation', 'DT_predicted_50 artifact',
                                              'DT_predicted_75 mutation', 'DT_predicted_75 artifact',
                                              'DT_predicted_90 mutation', 'DT_predicted_90 artifact',
                                              'DT_predicted_95 mutation', 'DT_predicted_95 artifact',
                                              'DT_predicted_97.5 mutation', 'DT_predicted_97.5 artifact',
                                              'DT_predicted_99 mutation', 'DT_predicted_99 artifact'],
                                       columns=['MicroSEC mutation', 'MicroSEC artifact'])

y_pred_temp_25 = copy.deepcopy(y_proba_total_ADA)
for i in range(len(y_proba_total_ADA)):
    if y_proba_total_ADA[i]>=0.25:
        y_pred_temp_25[i]=1
    else:
        y_pred_temp_25[i]=0

y_pred_temp_50 = copy.deepcopy(y_proba_total_ADA)
for i in range(len(y_proba_total_ADA)):
    if y_proba_total_ADA[i]>=0.5:
        y_pred_temp_50[i]=1
    else:
        y_pred_temp_50[i]=0

y_pred_temp_75 = copy.deepcopy(y_proba_total_ADA)
for i in range(len(y_proba_total_ADA)):
    if y_proba_total_ADA[i]>=0.75:
        y_pred_temp_75[i]=1
    else:
        y_pred_temp_75[i]=0

y_pred_temp_90 = copy.deepcopy(y_proba_total_ADA)
for i in range(len(y_proba_total_ADA)):
    if y_proba_total_ADA[i]>=0.9:
        y_pred_temp_90[i]=1
    else:
        y_pred_temp_90[i]=0

y_pred_temp_95 = copy.deepcopy(y_proba_total_ADA)
for i in range(len(y_proba_total_ADA)):
    if y_proba_total_ADA[i]>=0.95:
        y_pred_temp_95[i]=1
    else:
        y_pred_temp_95[i]=0

y_pred_temp_975 = copy.deepcopy(y_proba_total_ADA)
for i in range(len(y_proba_total_ADA)):
    if y_proba_total_ADA[i]>=0.975:
        y_pred_temp_975[i]=1
    else:
        y_pred_temp_975[i]=0

y_pred_temp_99 = copy.deepcopy(y_proba_total_ADA)
for i in range(len(y_proba_total_ADA)):
    if y_proba_total_ADA[i]>=0.99:
        y_pred_temp_99[i]=1
    else:
        y_pred_temp_99[i]=0

conf_matrix_ADA = confusion_matrix(y_pred_temp_25, y_true_ADA)
conf_matrix_ADA = np.concatenate([conf_matrix_ADA, confusion_matrix(y_pred_temp_50, y_true_ADA)])
conf_matrix_ADA = np.concatenate([conf_matrix_ADA, confusion_matrix(y_pred_temp_75, y_true_ADA)])
conf_matrix_ADA = np.concatenate([conf_matrix_ADA, confusion_matrix(y_pred_temp_90, y_true_ADA)])
conf_matrix_ADA = np.concatenate([conf_matrix_ADA, confusion_matrix(y_pred_temp_95, y_true_ADA)])
conf_matrix_ADA = np.concatenate([conf_matrix_ADA, confusion_matrix(y_pred_temp_975, y_true_ADA)])
conf_matrix_ADA = np.concatenate([conf_matrix_ADA, confusion_matrix(y_pred_temp_99, y_true_ADA)])

conf_table_ADA = pd.DataFrame(conf_matrix_ADA, index=['ADA_predicted_25 mutation', 'ADA_predicted_25 artifact', 
                                              'ADA_predicted_50 mutation', 'ADA_predicted_50 artifact',
                                              'ADA_predicted_75 mutation', 'ADA_predicted_75 artifact',
                                              'ADA_predicted_90 mutation', 'ADA_predicted_90 artifact',
                                              'ADA_predicted_95 mutation', 'ADA_predicted_95 artifact',
                                              'ADA_predicted_97.5 mutation', 'ADA_predicted_97.5 artifact',
                                              'ADA_predicted_99 mutation', 'ADA_predicted_99 artifact'],
                                       columns=['MicroSEC mutation', 'MicroSEC artifact'])
pd.concat([conf_table_DT, conf_table_ADA], axis=0)



In [ ]:
# Naive Bayes
# 5-fold cross validation
# Training:Validation:Test = 3:1:1

X_GNB = np.array(X)
y_GNB = np.array(y)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_pred_total_GNB = []
y_proba_total_GNB = []
y_true_GNB = []
for train, test in skf.split(X_GNB, y_GNB):
    X_train = X_GNB[train]
    X_test = X_GNB[test]
    y_train = y_GNB[train]
    y_test = y_GNB[test]
    #X_train, X_valid, y_train, y_valid = train_test_split(X_tv, y_tv, shuffle=True, test_size=0.25, stratify=y_tv, random_state=100)

    search_params = {'var_smoothing': np.logspace(0,-9, num=100)}
    GNB = GridSearchCV(GaussianNB(),
                      search_params,
                      cv=4,
                      verbose=True,
                      scoring='neg_log_loss',
                      n_jobs=-1)
    GNB.fit(X_train, y_train)

    print(GNB.best_estimator_)
    print(f"acc: {GNB.score(X_test, y_test)}")
    y_pred = GNB.predict(X_test)
    y_proba = GNB.predict_proba(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1])
    roc_auc = auc(fpr, tpr)
    print ("AUC curve : %f" % roc_auc)
    plt.figure(figsize=(4,4))
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve: AUC=%0.2f' % roc_auc)
    plt.legend(loc="lower right")
    plt.show()
    y_pred_total_GNB = np.concatenate([y_pred_total_GNB, y_pred])
    y_proba_total_GNB = np.concatenate([y_proba_total_GNB, y_proba[:, 1]])
    y_true_GNB = np.concatenate([y_true_GNB, y_test])

fpr, tpr, thresholds = roc_curve(y_true_GNB, y_proba_total_GNB)
roc_auc = auc(fpr, tpr)
print ("AUC curve : %f" % roc_auc)
plt.figure(figsize=(4,4))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve: AUC=%0.2f' % roc_auc)
plt.legend(loc="lower right")
plt.show()

y_pred_temp_25 = copy.deepcopy(y_proba_total_GNB)
for i in range(len(y_proba_total_GNB)):
    if y_proba_total_GNB[i]>=0.25:
        y_pred_temp_25[i]=1
    else:
        y_pred_temp_25[i]=0

y_pred_temp_50 = copy.deepcopy(y_proba_total_GNB)
for i in range(len(y_proba_total_GNB)):
    if y_proba_total_GNB[i]>=0.5:
        y_pred_temp_50[i]=1
    else:
        y_pred_temp_50[i]=0

y_pred_temp_75 = copy.deepcopy(y_proba_total_GNB)
for i in range(len(y_proba_total_GNB)):
    if y_proba_total_GNB[i]>=0.75:
        y_pred_temp_75[i]=1
    else:
        y_pred_temp_75[i]=0

y_pred_temp_90 = copy.deepcopy(y_proba_total_GNB)
for i in range(len(y_proba_total_GNB)):
    if y_proba_total_GNB[i]>=0.9:
        y_pred_temp_90[i]=1
    else:
        y_pred_temp_90[i]=0

y_pred_temp_95 = copy.deepcopy(y_proba_total_GNB)
for i in range(len(y_proba_total_GNB)):
    if y_proba_total_GNB[i]>=0.95:
        y_pred_temp_95[i]=1
    else:
        y_pred_temp_95[i]=0

y_pred_temp_975 = copy.deepcopy(y_proba_total_GNB)
for i in range(len(y_proba_total_GNB)):
    if y_proba_total_GNB[i]>=0.975:
        y_pred_temp_975[i]=1
    else:
        y_pred_temp_975[i]=0

y_pred_temp_99 = copy.deepcopy(y_proba_total_GNB)
for i in range(len(y_proba_total_GNB)):
    if y_proba_total_GNB[i]>=0.99:
        y_pred_temp_99[i]=1
    else:
        y_pred_temp_99[i]=0

conf_matrix_GNB = confusion_matrix(y_pred_temp_25, y_true_GNB)
conf_matrix_GNB = np.concatenate([conf_matrix_GNB, confusion_matrix(y_pred_temp_50, y_true_GNB)])
conf_matrix_GNB = np.concatenate([conf_matrix_GNB, confusion_matrix(y_pred_temp_75, y_true_GNB)])
conf_matrix_GNB = np.concatenate([conf_matrix_GNB, confusion_matrix(y_pred_temp_90, y_true_GNB)])
conf_matrix_GNB = np.concatenate([conf_matrix_GNB, confusion_matrix(y_pred_temp_95, y_true_GNB)])
conf_matrix_GNB = np.concatenate([conf_matrix_GNB, confusion_matrix(y_pred_temp_975, y_true_GNB)])
conf_matrix_GNB = np.concatenate([conf_matrix_GNB, confusion_matrix(y_pred_temp_99, y_true_GNB)])

conf_table_GNB = pd.DataFrame(conf_matrix_GNB, index=['GNB_predicted_25 mutation', 'GNB_predicted_25 artifact', 
                                              'GNB_predicted_50 mutation', 'GNB_predicted_50 artifact',
                                              'GNB_predicted_75 mutation', 'GNB_predicted_75 artifact',
                                              'GNB_predicted_90 mutation', 'GNB_predicted_90 artifact',
                                              'GNB_predicted_95 mutation', 'GNB_predicted_95 artifact',
                                              'GNB_predicted_97.5 mutation', 'GNB_predicted_97.5 artifact',
                                              'GNB_predicted_99 mutation', 'GNB_predicted_99 artifact'],
                                       columns=['MicroSEC mutation', 'MicroSEC artifact'])

conf_table_GNB

In [ ]:
# MSK impact re-analysis with Random forest clasi

gs = GridSearchCV(RFC(),
                  search_params,
                  cv=4,
                  verbose=False,
                  n_jobs=-1)
gs.fit(X_RF, y_RF)

print(gs.best_estimator_)

y_MSK_proba = gs.predict_proba(X_MSK)[:,1]
df_MSK[y_MSK_proba>0.95]

In [ ]:
# Light GBM
# 5-fold cross validation
# Training:Validation:Test = 3:1:1

X_LGB = np.array(X)
y_LGB = np.array(y)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_pred_total_LGB = []
y_proba_total_LGB = []
y_true_LGB = []
for train, test in skf.split(X_LGB, y_LGB):
    X_train = X_LGB[train]
    X_test = X_LGB[test]
    y_train = y_LGB[train]
    y_test = y_LGB[test]

    lgbm = lgb.LGBMClassifier(objective='binary')
    
    search_params = {'learning_rate':[0.01],
                     'metric':['binary_logloss'],
                     'num_iteration':[4000],
                     'num_leaves':[3, 4, 5, 6, 7, 8, 9, 10],
                     'reg_alpha':[0, 1, 2, 3, 4, 5, 10, 100],
                     'reg_lambda':[10, 15, 18, 20, 21, 22, 23, 25, 27, 29]
                    }

    LGB = GridSearchCV(lgbm,
                      search_params,
                      cv=4,
                      verbose=True,
                      scoring='neg_log_loss',
                      n_jobs=-1)
    LGB.fit(X_train, y_train)

    print(LGB.best_estimator_)
    print(f"acc: {LGB.score(X_test, y_test)}")
    y_pred = LGB.predict(X_test)
    y_proba = LGB.predict_proba(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 1])
    roc_auc = auc(fpr, tpr)
    print ("AUC curve : %f" % roc_auc)
    plt.figure(figsize=(4,4))
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve: AUC=%0.2f' % roc_auc)
    plt.legend(loc="lower right")
    plt.show()
    y_pred_total_LGB = np.concatenate([y_pred_total_LGB, y_pred])
    y_proba_total_LGB = np.concatenate([y_proba_total_LGB, y_proba[:, 1]])
    y_true_LGB = np.concatenate([y_true_LGB, y_test])

fpr, tpr, thresholds = roc_curve(y_true_LGB, y_proba_total_LGB)
roc_auc = auc(fpr, tpr)
print ("AUC curve : %f" % roc_auc)
plt.figure(figsize=(4,4))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve: AUC=%0.2f' % roc_auc)
plt.legend(loc="lower right")
plt.show()

y_pred_temp_25 = copy.deepcopy(y_proba_total_LGB)
for i in range(len(y_proba_total_LGB)):
    if y_proba_total_LGB[i]>=0.25:
        y_pred_temp_25[i]=1
    else:
        y_pred_temp_25[i]=0

y_pred_temp_50 = copy.deepcopy(y_proba_total_LGB)
for i in range(len(y_proba_total_LGB)):
    if y_proba_total_LGB[i]>=0.5:
        y_pred_temp_50[i]=1
    else:
        y_pred_temp_50[i]=0

y_pred_temp_75 = copy.deepcopy(y_proba_total_LGB)
for i in range(len(y_proba_total_LGB)):
    if y_proba_total_LGB[i]>=0.75:
        y_pred_temp_75[i]=1
    else:
        y_pred_temp_75[i]=0

y_pred_temp_90 = copy.deepcopy(y_proba_total_LGB)
for i in range(len(y_proba_total_LGB)):
    if y_proba_total_LGB[i]>=0.9:
        y_pred_temp_90[i]=1
    else:
        y_pred_temp_90[i]=0

y_pred_temp_95 = copy.deepcopy(y_proba_total_LGB)
for i in range(len(y_proba_total_LGB)):
    if y_proba_total_LGB[i]>=0.95:
        y_pred_temp_95[i]=1
    else:
        y_pred_temp_95[i]=0

y_pred_temp_975 = copy.deepcopy(y_proba_total_LGB)
for i in range(len(y_proba_total_LGB)):
    if y_proba_total_LGB[i]>=0.975:
        y_pred_temp_975[i]=1
    else:
        y_pred_temp_975[i]=0

y_pred_temp_99 = copy.deepcopy(y_proba_total_LGB)
for i in range(len(y_proba_total_LGB)):
    if y_proba_total_LGB[i]>=0.99:
        y_pred_temp_99[i]=1
    else:
        y_pred_temp_99[i]=0

conf_matrix_LGB = confusion_matrix(y_pred_temp_25, y_true_LGB)
conf_matrix_LGB = np.concatenate([conf_matrix_LGB, confusion_matrix(y_pred_temp_50, y_true_LGB)])
conf_matrix_LGB = np.concatenate([conf_matrix_LGB, confusion_matrix(y_pred_temp_75, y_true_LGB)])
conf_matrix_LGB = np.concatenate([conf_matrix_LGB, confusion_matrix(y_pred_temp_90, y_true_LGB)])
conf_matrix_LGB = np.concatenate([conf_matrix_LGB, confusion_matrix(y_pred_temp_95, y_true_LGB)])
conf_matrix_LGB = np.concatenate([conf_matrix_LGB, confusion_matrix(y_pred_temp_975, y_true_LGB)])
conf_matrix_LGB = np.concatenate([conf_matrix_LGB, confusion_matrix(y_pred_temp_99, y_true_LGB)])

conf_table_LGB = pd.DataFrame(conf_matrix_LGB, index=['LGB_predicted_25 mutation', 'LGB_predicted_25 artifact', 
                                              'LGB_predicted_50 mutation', 'LGB_predicted_50 artifact',
                                              'LGB_predicted_75 mutation', 'LGB_predicted_75 artifact',
                                              'LGB_predicted_90 mutation', 'LGB_predicted_90 artifact',
                                              'LGB_predicted_95 mutation', 'LGB_predicted_95 artifact',
                                              'LGB_predicted_97.5 mutation', 'LGB_predicted_97.5 artifact',
                                              'LGB_predicted_99 mutation', 'LGB_predicted_99 artifact'],
                                       columns=['MicroSEC mutation', 'MicroSEC artifact'])

conf_table_LGB

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true_LGB, y_pred_total_LGB)
roc_auc = auc(fpr, tpr)
label = 'LightGBM' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)

fpr, tpr, thresholds = roc_curve(y_true_GNB, y_pred_total_GNB)
roc_auc = auc(fpr, tpr)
label = 'Naive Bayes' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)

fpr, tpr, thresholds = roc_curve(y_true_LR, y_pred_total_LR)
roc_auc = auc(fpr, tpr)
label = 'Logistic regression' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)

fpr, tpr, thresholds = roc_curve(y_true_DT, y_pred_total_DT)
roc_auc = auc(fpr, tpr)
label = 'Decision tree classifier' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)

fpr, tpr, thresholds = roc_curve(y_true_ADA, y_pred_total_ADA)
roc_auc = auc(fpr, tpr)
label = 'ADABoost' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)


fpr, tpr, thresholds = roc_curve(y_true, y_pred_total)
roc_auc = auc(fpr, tpr)
label = 'Random forest classifier' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)





plt.figure(figsize=(10,10))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curves')
plt.legend(loc="lower right")
plt.show()

fpr, tpr, thresholds = roc_curve(y_true, y_proba_total)
roc_auc = auc(fpr, tpr)
print ("Random forest classifier AUC curve : %f" % roc_auc)
plt.figure(figsize=(4,4))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve: AUC=%0.2f' % roc_auc)
plt.legend(loc="lower right")
plt.show()

pd.concat([conf_table, conf_table_LGB], axis=0)

In [ ]:
print(model.feature_importance())
lgb.plot_importance(model, height = 0.5, figsize = (8,16))

importance = pd.DataFrame(model.feature_importance(), index = X.columns, columns=["importance"])
display(importance.sort_values("importance", ascending= False))
importance.sum()

In [ ]:
y_proba = model.predict(X_MSK)
df_MSK[y_proba>0.99]

In [ ]:
importance.sort_values("importance", ascending= True).plot.barh(figsize = (8,16))

In [ ]:
importance.sort_values("importance", ascending= False)

In [ ]:
# Neural network
# 5-fold cross validation
# Training:Validation:Test = 3:1:1
    
X_NN = np.array(X)
y_NN = np.array(y)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
y_pred_total_NN = []
y_proba_total_NN = []
y_true_NN = []
for train, test in skf.split(X_NN, y_NN):
    X_tv = X_NN[train]
    X_test = X_NN[test]
    y_tv = y_NN[train]
    y_test = y_NN[test]
    X_train, X_valid, y_train, y_valid = train_test_split(X_tv, y_tv, shuffle=True, test_size=0.25, stratify=y_tv, random_state=100)

    inputs = keras.Input(shape=(72,), name='img')
    x = layers.Dense(1024, activation='relu')(inputs)
    x = layers.Dense(256, activation='relu')(inputs)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(16, activation='relu')(x)
    outputs = layers.Dense(2, activation='softmax')(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name='NN_model')

    model.compile(loss='BinaryCrossentropy',
                  optimizer=keras.optimizers.RMSprop(),
                  metrics=['binary_accuracy'])
    early_stopping = EarlyStopping(patience=30, verbose=1) 
    history = model.fit(X_tv, y_tv,
                        batch_size=64,
                        epochs=500,
                        callbacks=[early_stopping],
                        validation_split=0.25)
    test_scores = model.evaluate(X_test, y_test, verbose=2)
    print('Test loss:', test_scores[0])
    print('Test accuracy:', test_scores[1])

    y_pred = model_functional.predict(X_test)
    y_proba = model_functional.predict(X_test)
    fpr, tpr, thresholds = roc_curve(y_test, y_proba[:, 0])
    roc_auc = auc(fpr, tpr)
    print ("AUC curve : %f" % roc_auc)
    plt.figure(figsize=(4,4))
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC curve: AUC=%0.2f' % roc_auc)
    plt.legend(loc="lower right")
    plt.show()
    y_pred_total_NN = np.concatenate([y_pred_total_NN, y_pred[:, 0]])
    y_proba_total_NN = np.concatenate([y_proba_total_NN, y_proba[:, 0]])
    y_true_NN = np.concatenate([y_true_NN, y_test])

fpr, tpr, thresholds = roc_curve(y_true_NN, y_proba_total_NN)
roc_auc = auc(fpr, tpr)
print ("AUC curve : %f" % roc_auc)
plt.figure(figsize=(4,4))
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve: AUC=%0.2f' % roc_auc)
plt.legend(loc="lower right")
plt.show()

y_pred_temp_25 = copy.deepcopy(y_proba_total_NN)
for i in range(len(y_proba_total_NN)):
    if y_proba_total_NN[i]>=0.25:
        y_pred_temp_25[i]=1
    else:
        y_pred_temp_25[i]=0

y_pred_temp_50 = copy.deepcopy(y_proba_total_NN)
for i in range(len(y_proba_total_NN)):
    if y_proba_total_NN[i]>=0.5:
        y_pred_temp_50[i]=1
    else:
        y_pred_temp_50[i]=0

y_pred_temp_75 = copy.deepcopy(y_proba_total_NN)
for i in range(len(y_proba_total_NN)):
    if y_proba_total_NN[i]>=0.75:
        y_pred_temp_75[i]=1
    else:
        y_pred_temp_75[i]=0

y_pred_temp_90 = copy.deepcopy(y_proba_total_NN)
for i in range(len(y_proba_total_NN)):
    if y_proba_total_NN[i]>=0.9:
        y_pred_temp_90[i]=1
    else:
        y_pred_temp_90[i]=0

y_pred_temp_95 = copy.deepcopy(y_proba_total_NN)
for i in range(len(y_proba_total_NN)):
    if y_proba_total_NN[i]>=0.95:
        y_pred_temp_95[i]=1
    else:
        y_pred_temp_95[i]=0

y_pred_temp_975 = copy.deepcopy(y_proba_total_NN)
for i in range(len(y_proba_total_NN)):
    if y_proba_total_NN[i]>=0.975:
        y_pred_temp_975[i]=1
    else:
        y_pred_temp_975[i]=0

y_pred_temp_99 = copy.deepcopy(y_proba_total_NN)
for i in range(len(y_proba_total_NN)):
    if y_proba_total_NN[i]>=0.99:
        y_pred_temp_99[i]=1
    else:
        y_pred_temp_99[i]=0

conf_matrix_NN = confusion_matrix(y_pred_temp_25, y_true_NN)
conf_matrix_NN = np.concatenate([conf_matrix_NN, confusion_matrix(y_pred_temp_50, y_true_NN)])
conf_matrix_NN = np.concatenate([conf_matrix_NN, confusion_matrix(y_pred_temp_75, y_true_NN)])
conf_matrix_NN = np.concatenate([conf_matrix_NN, confusion_matrix(y_pred_temp_90, y_true_NN)])
conf_matrix_NN = np.concatenate([conf_matrix_NN, confusion_matrix(y_pred_temp_95, y_true_NN)])
conf_matrix_NN = np.concatenate([conf_matrix_NN, confusion_matrix(y_pred_temp_975, y_true_NN)])
conf_matrix_NN = np.concatenate([conf_matrix_NN, confusion_matrix(y_pred_temp_99, y_true_NN)])

conf_table_NN = pd.DataFrame(conf_matrix_NN, index=['NN_predicted_25 mutation', 'NN_predicted_25 artifact', 
                                              'NN_predicted_50 mutation', 'NN_predicted_50 artifact',
                                              'NN_predicted_75 mutation', 'NN_predicted_75 artifact',
                                              'NN_predicted_90 mutation', 'NN_predicted_90 artifact',
                                              'NN_predicted_95 mutation', 'NN_predicted_95 artifact',
                                              'NN_predicted_97.5 mutation', 'NN_predicted_97.5 artifact',
                                              'NN_predicted_99 mutation', 'NN_predicted_99 artifact'],
                                       columns=['MicroSEC mutation', 'MicroSEC artifact'])

conf_table_NN

In [ ]:
plt.figure(figsize=(10,10))

fpr, tpr, thresholds = roc_curve(y_true_LGB, y_proba_total_LGB)
roc_auc = auc(fpr, tpr)
label = 'LightGBM' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)

fpr, tpr, thresholds = roc_curve(y_true_GNB, y_proba_total_GNB)
roc_auc = auc(fpr, tpr)
label = 'Naive Bayes' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)

fpr, tpr, thresholds = roc_curve(y_true_LR, y_proba_total_LR)
roc_auc = auc(fpr, tpr)
label = 'Logistic regression' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)

fpr, tpr, thresholds = roc_curve(y_true_DT, y_proba_total_DT)
roc_auc = auc(fpr, tpr)
label = 'Decision tree classifier' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)

fpr, tpr, thresholds = roc_curve(y_true_ADA, y_proba_total_ADA)
roc_auc = auc(fpr, tpr)
label = 'ADABoost' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)

fpr, tpr, thresholds = roc_curve(y_true, y_proba_total)
roc_auc = auc(fpr, tpr)
label = 'Random forest classifier' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)


plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 0.2])
plt.ylim([0.8, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curves')
plt.legend(loc="lower right")
plt.show()

plt.figure(figsize=(10,10))

fpr, tpr, thresholds = roc_curve(y_true_LGB, y_proba_total_LGB)
roc_auc = auc(fpr, tpr)
label = 'LightGBM' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)

fpr, tpr, thresholds = roc_curve(y_true_GNB, y_proba_total_GNB)
roc_auc = auc(fpr, tpr)
label = 'Naive Bayes' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)

fpr, tpr, thresholds = roc_curve(y_true_LR, y_proba_total_LR)
roc_auc = auc(fpr, tpr)
label = 'Logistic regression' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)

fpr, tpr, thresholds = roc_curve(y_true_DT, y_proba_total_DT)
roc_auc = auc(fpr, tpr)
label = 'Decision tree classifier' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)

fpr, tpr, thresholds = roc_curve(y_true_ADA, y_proba_total_ADA)
roc_auc = auc(fpr, tpr)
label = 'ADABoost' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)

fpr, tpr, thresholds = roc_curve(y_true, y_proba_total)
roc_auc = auc(fpr, tpr)
label = 'Random forest classifier' + ' (AUC={:.3f})'.format(auc(fpr, tpr))
plt.plot(fpr, tpr, label=label)


plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curves')
plt.legend(loc="lower right")
plt.show()




In [ ]:
X_DT.shape